In [1]:
import pickle
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
import pandas as pd

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
train_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/noun/train_hypernyms.pickle'
with open(train_path, 'rb') as f:
    train = pickle.load(f)

test_path = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/noun/test_hypernyms.pickle'
with open(test_path, 'rb') as f:
    test = pickle.load(f)

In [41]:
train[0], test[10]

({'children': 'dancing.n.01',
  'parents': 'diversion.n.01',
  'grandparents': None,
  'case': 'predict_hypernym'},
 {'children': 'streetcar.n.01',
  'grandparents': None,
  'parents': 'self-propelled_vehicle.n.01',
  'case': 'predict_hypernym'})

In [42]:
def add_definitions(elem):
    if elem['case'] == 'predict_hypernym':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
      #  elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'predict_multiple_hypernyms':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()

    elif elem['case'] == 'simple_triplet_grandparent':
        elem['child_def'] = wn.synset(elem['children'].replace(' ', '_')).definition()
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
    elif elem['case'] == 'only_child_leaf':
        elem['grandparent_def'] = wn.synset(elem['grandparents']).definition()
        elem['parent_def'] = wn.synset(elem['parents']).definition()
    elif elem['case'] == 'simple_triplet_2parent':
        elem['1parent_def'] = wn.synset(elem['parents'][0]).definition()
        elem['2parent_def'] = wn.synset(elem['parents'][1]).definition()
    else:
        elem['parent_def'] = wn.synset(elem['parents']).definition()

def cut_lemmas(lemmas):
    max_lemmas = min(len(lemmas), 8)
    return lemmas[:max_lemmas]

max_len = 0
def add_lemmas(elem):
    global max_len
    if elem['case'] == 'predict_hypernym':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_grandparent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['children'].replace(' ', '_')).lemmas()])
        elem['child_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'only_child_leaf':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['grandparents']).lemmas()])
        elem['grandparent_def'] = ', '.join(lemmas)

        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    elif elem['case'] == 'simple_triplet_2parent':
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][0]).lemmas()])
        elem['1parent_def'] = ', '.join(lemmas)
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents'][1]).lemmas()])
        elem['2parent_def'] = ', '.join(lemmas)
    else:
        lemmas = cut_lemmas([str(lemma.name()) for lemma in wn.synset(elem['parents']).lemmas()])
        elem['parent_def'] = ', '.join(lemmas)
    if len(lemmas) > max_len:
        max_len = len(lemmas)

In [43]:
for i, elem in enumerate(train):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
       # train.remove(elem)

In [44]:
i

41362

In [45]:
test[37]

{'children': 'test.test.66',
 'grandparents': None,
 'parents': 'ablation.n.01',
 'case': 'predict_hypernym'}

In [46]:
counter = 0
for i, elem in enumerate(test):
    try:
        add_definitions(elem)
    except:
        print(i, elem)
        counter += 1
        test.remove(elem)

print(counter)

37 {'children': 'test.test.66', 'grandparents': None, 'parents': 'ablation.n.01', 'case': 'predict_hypernym'}
68 {'children': 'test.test.276', 'grandparents': None, 'parents': 'anchor.n.01', 'case': 'predict_hypernym'}
93 {'children': 'test.test.485', 'grandparents': None, 'parents': 'security_system.n.01', 'case': 'predict_hypernym'}
161 {'children': 'val.withdef.114', 'grandparents': None, 'parents': 'sport.n.01', 'case': 'predict_hypernym'}
222 {'children': 'train.withdef.153', 'grandparents': None, 'parents': 'debris.n.01', 'case': 'predict_hypernym'}
293 {'children': 'val.withdef.86', 'grandparents': None, 'parents': 'broadcast.n.02', 'case': 'predict_hypernym'}
333 {'children': 'train.withdef.160', 'grandparents': None, 'parents': 'erosion.n.01', 'case': 'predict_hypernym'}
534 {'children': 'train.withdef.52', 'grandparents': None, 'parents': 'increase.n.05', 'case': 'predict_hypernym'}
670 {'children': 'train.withdef.119', 'grandparents': None, 'parents': 'program.n.07', 'case':

In [48]:
len(test), len(train)

(986, 41363)

In [50]:
train_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/noun/train_hypernyms_def.pickle'
test_out = '/home/LLM_Taxonomy/TaxonomyEnrichment/data/noun/test_hypernyms_def.picklee'

with open(train_out, 'wb') as f:
    pickle.dump(train, f)

with open(test_out, 'wb') as f:
    pickle.dump(test, f)

In [6]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')
from pipeline_src.dataset.prompt_schemas import (
    hypo_term_hyper,
    predict_child_from_2_parents,
    predict_child_from_parent,
    predict_child_with_parent_and_grandparent,
    predict_children_with_parent_and_brothers,
    predict_parent_from_child_granparent,
    predict_parent_from_child,
)

In [5]:
def clean_elem(elem, keys_to_remove_digits=["children"]):
    removes = set(keys_to_remove_digits)
    if not "changed" in elem.keys():
        for field in ["children", "parents", "grandparents", "brothers"]:
            if field in elem.keys():
                elem[field] = delete_techniqal(
                    elem[field], remove=(field in removes))
                elem["changed"] = True
    return elem


def delete_techniqal(elem, remove):
    if isinstance(elem, str):
        if ".n." in elem and remove:
            return elem.split(".")[0].replace("_", " ")
        else:
            return elem.replace("_", " ")

    elif isinstance(elem, list):
        new_words = []
        for word in elem:
            new_words.append(delete_techniqal(word, remove))
        return new_words

def predict_parent_from_child(elem):
    """
    Predict the hypernym for the word “spaniel”. Answer: (sporting dog)
    """
    keys_to_remove_digits = []
    if use_def_prompt:
        keys_to_remove_digits.append("children") # do not need numbers when provided definition
    if (not use_number_target) or (use_def_target):
        keys_to_remove_digits.append("parents")
    
    clean = clean_elem(elem, keys_to_remove_digits=keys_to_remove_digits)

    if use_def_prompt:
        transformed_term = (
            "hyponym: " + clean["children"] + " (" + elem["child_def"] + ") | hypernym:"
        )
    else:
        transformed_term = "hyponym: " + clean["children"] + " | hypernym:"

    if use_def_target:
        target = clean["parents"] + " (" + elem["parent_def"] + ")" + ","
    else:
        target = clean["parents"] + ","

    return transformed_term, target

In [19]:
te = dict(elem)

In [20]:
use_def_prompt = True
use_def_target = True
use_number_target = True

predict_parent_from_child(te)

('hyponym: light hour (the distance light travels in a vacuum in one hour; approximately one billion kilometers) | hypernym:',
 'astronomy unit (a linear unit used for astronomical distances),')

In [9]:
processed_term

'hypernym: optical instrument (an instrument designed to aid vision) | hyponyms:'